In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# Base URL
base_url = "https://scrapeme.live/shop/"

# Lists to store product data
names = []
prices = []
descriptions = []
categories = []
skus = []
tags_list = []
stocks = []

In [3]:
# Request homepage to find how many pages exist
response = requests.get(base_url, verify=False)
soup = BeautifulSoup(response.text, "html.parser")

# Get last page number
pages = soup.find_all("a", class_="page-numbers")
last_page = int(pages[-2].text) if pages else 1

print(f"Total pages found: {last_page}")

Total pages found: 48


In [6]:
response

<Response [200]>

In [7]:
response.text

'\n<!doctype html>\n<html lang="en-GB">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=2.0">\n<link rel="profile" href="http://gmpg.org/xfn/11">\n<link rel="pingback" href="https://scrapeme.live/xmlrpc.php">\n\n<title>Products &#8211; Page 48 &#8211; ScrapeMe</title>\n<link rel=\'dns-prefetch\' href=\'//fonts.googleapis.com\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel="alternate" type="application/rss+xml" title="ScrapeMe &raquo; Feed" href="https://scrapeme.live/feed/" />\n<link rel="alternate" type="application/rss+xml" title="ScrapeMe &raquo; Comments Feed" href="https://scrapeme.live/comments/feed/" />\n<link rel="alternate" type="application/rss+xml" title="ScrapeMe &raquo; Products Feed" href="https://scrapeme.live/shop/feed/" />\n\t\t<script type="text/javascript">\n\t\t\twindow._wpemojiSettings = {"baseUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/11\\/72x72\\/","ext":".png","svgUrl":

In [4]:
# Loop through every page
for page in range(1, last_page + 1):
    url = base_url if page == 1 else f"{base_url}page/{page}/"
    print(f"Scraping page {page} ...")

    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all products in page
    products = soup.find_all("li", class_="product")

    for product in products:
        # Product name
        name = product.find("h2", class_="woocommerce-loop-product__title")
        name = name.text.strip() if name else ""

        # Product price
        price = product.find("span", class_="woocommerce-Price-amount")
        price = price.text.strip() if price else ""

        # Link to detail page
        link = product.find("a", href=True)
        if link:
            detail_url = link['href']
            detail_response = requests.get(detail_url, verify=False)
            detail_soup = BeautifulSoup(detail_response.text, "html.parser")

            # Description
            desc_tag = detail_soup.find("div", class_="woocommerce-product-details__short-description")
            description = desc_tag.text.strip() if desc_tag else ""

            # Category
            cat_tag = detail_soup.find("span", class_="posted_in")
            category = cat_tag.text.replace("Category:", "").replace("Categories:", "").strip() if cat_tag else ""

            # SKU
            sku_tag = detail_soup.find("span", class_="sku")
            sku = sku_tag.text.strip() if sku_tag else ""

            # Tags
            tag_tag = detail_soup.find("span", class_="tagged_as")
            tags = tag_tag.text.replace("Tags:", "").strip() if tag_tag else ""

            # Stock status
            stock_tag = detail_soup.find("p", class_="stock")
            stock = stock_tag.text.strip() if stock_tag else ""
        else:
            description, category, sku, tags, stock = "", "", "", "", ""

        # Append to lists
        names.append(name)
        prices.append(price)
        descriptions.append(description)
        categories.append(category)
        skus.append(sku)
        tags_list.append(tags)
        stocks.append(stock)

Scraping page 1 ...
Scraping page 2 ...
Scraping page 3 ...
Scraping page 4 ...
Scraping page 5 ...
Scraping page 6 ...
Scraping page 7 ...
Scraping page 8 ...
Scraping page 9 ...
Scraping page 10 ...
Scraping page 11 ...
Scraping page 12 ...
Scraping page 13 ...
Scraping page 14 ...
Scraping page 15 ...
Scraping page 16 ...
Scraping page 17 ...
Scraping page 18 ...
Scraping page 19 ...
Scraping page 20 ...
Scraping page 21 ...
Scraping page 22 ...
Scraping page 23 ...
Scraping page 24 ...
Scraping page 25 ...
Scraping page 26 ...
Scraping page 27 ...
Scraping page 28 ...
Scraping page 29 ...
Scraping page 30 ...
Scraping page 31 ...
Scraping page 32 ...
Scraping page 33 ...
Scraping page 34 ...
Scraping page 35 ...
Scraping page 36 ...
Scraping page 37 ...
Scraping page 38 ...
Scraping page 39 ...
Scraping page 40 ...
Scraping page 41 ...
Scraping page 42 ...
Scraping page 43 ...
Scraping page 44 ...
Scraping page 45 ...
Scraping page 46 ...
Scraping page 47 ...
Scraping page 48 ...


In [5]:
# Create DataFrame
df = pd.DataFrame({
    "Name": names,
    "Price": prices,
    "Description": descriptions,
    "Category": categories,
    "SKU": skus,
    "Tags": tags_list,
    "Stock_Status": stocks
})

# Save to CSV
df.to_csv("scrapeme_products_full.csv", index=False)

print(df.head())
print(f"\nTotal products scraped: {len(df)}")
print("Data saved to scrapeme_products_full.csv")

         Name    Price                                        Description  \
0   Bulbasaur   £63.00  Bulbasaur can be seen napping in bright sunlig...   
1     Ivysaur   £87.00  There is a bud on this Pokémon’s back. To supp...   
2    Venusaur  £105.00  There is a large flower on Venusaur’s back. Th...   
3  Charmander   £48.00  The flame that burns at the tip of its tail is...   
4  Charmeleon  £165.00  Charmeleon mercilessly destroys its foes using...   

          Category   SKU                       Tags  Stock_Status  
0    Pokemon, Seed  4391  bulbasaur, Overgrow, Seed   45 in stock  
1    Pokemon, Seed  7227    ivysaur, Overgrow, Seed  142 in stock  
2    Pokemon, Seed  7036   Overgrow, Seed, venusaur   30 in stock  
3  Lizard, Pokemon  9086  Blaze, charmander, Lizard  206 in stock  
4   Flame, Pokemon  6565   Blaze, charmeleon, Flame  284 in stock  

Total products scraped: 755
Data saved to scrapeme_products_full.csv
